<a href="https://colab.research.google.com/github/geomyll33/Beautez/blob/master/Multiple_TIme_series_Forecasting_DeepAR_GluonTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade mxnet==1.6.0
!pip install gluonts

     |████████████████████████████████| 68.7MB 67kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 573kB 5.0MB/s 
     |████████████████████████████████| 8.6MB 25.9MB/s 
     |████████████████████████████████| 194kB 32.5MB/s 
  Created wheel for ujson: filename=ujson-1.35-cp37-cp37m-linux_x86_64.whl size=68391 sha256=b805d046c9f1af22d3d3d72bb87fc8c47ae272eddc85e2ce3126a91b42da9ccb
  Stored in directory: /root/.cache/pip/wheels/28/77/e4/0311145b9c2e2f01470e744855131f9e34d6919687550f87d1
Successfully built ujson


In [4]:
import pandas as pd
import matplotlib.pyplot as mpl
import matplotlib.pyplot as plt
from gluonts.model.deepar import DeepAREstimator
from gluonts.trainer import Trainer
import numpy as np
mpl.rcParams['figure.figsize']=(10,8)
mpl.rcParams['axes.grid']=False

In [ ]:
fig,axs=plt.subplots(5,2,figsize=(20,20),sharex=True)
axx=axs.ravel()
for i in range(0,10):
  df[df.columns[i]].loc['2014-12-01':'2014-12-14'].plot(ax=axx[i])
  axx[i].set_xlabel('date')
  axx[i].set_ylabel('KW consuption')
  axx[i].grid(which='minor',axis='x')

In [ ]:
df=df.reset_index(drop=True).T.reset_index()#raws to columns

In [ ]:
ts_code=df['index'].astype('category').cat.codes.values#tranform index to categorical labels

In [ ]:
df_train=df.iloc[:,1:134999].values#shape is 370X140257 do not shuffle!!
df_test=df.iloc[:,134999:].values

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
freq='15'
start_train=pd.Timestamp('2011-01-01 00:15:00', freq=freq)
start_test=pd.Timestamp('2014-11-07 05:30:00',freq=freq)
prediction=672

In [ ]:
estimator=DeepAREstimator(freq=freq,
                          context_length=672,
                          prediction_length=prediction_length,
                          use_feat_static_cat=True,cardinality=[1],
                          num_layers=2,
                          num_cells=32,
                          cell_type='lstm',
                          trainer=Trainer(epochs=5))

In [ ]:
df_train[0:7]

In [ ]:
from gluonts.dataset.common import ListDataset
from gluonts.dataset.field_names import FieldName

train_ds=ListDataset([
                      {
                       FieldName.TARGET:target,
                       FieldName.START:start_train
                       FieldName.FEAT_STATIC_CAT:fsc
                      }
                      for (target,fsc)in zip(df_train[0:7],
                                             ts_code[0:7].reshape(-1,1))
                      ],freq=freq)

test_ds=ListDataset([
                     {
                          FieldName.TARGET:target,
                       FieldName.START:start_test
                       FieldName.FEAT_STATIC_CAT:fsc
                      }
                      for (target,fsc)in zip(df_test[0:7],
                                             ts_code[0:7].reshape(-1,1))
                      ],freq=freq)
                     

In [ ]:
next(iter(train_ds))

In [ ]:
predictor=estimator.train(training_data=train_ds)

In [ ]:
from gluonts.evaluation.backtest import make_evaluation_predictions

In [ ]:
forecast_it,ts_it=make_evaluation_predictions(
    dataset=test_ds,
    predictor=predictor,
    num_samples=100,
)

In [ ]:
from tqdm.autonotebook import tqdm

print('Obtaining time series conditionong values...')
tss=list(tqdm(ts_it,total=len(df_test)))
print('Obtaining time series predictions...')
forecasts=list(tqdm(forecast_it,total=len(df_test)))

In [ ]:
def plot_prob_forecasts(ts_entry, forecast_entry):
  plot_length=prediction_length
  prediction_intrvals=(80.0,95.0)
  legend=['observations','median prediction']+[f'{k}% prediction interval' for k in prediction_intervals][::-1]

  fig,ax=plt.sublots(1,1,figsize=(10,7))
  ts_entry[-plot_length:].plt(ax=ax)
  forecast_entry.plot(prediction_intrvals=prediction_intervals,color='g')
  plt.grid(which='both')
  plt.legend(legend,loc='upper left')
  plt.show()

In [ ]:
for i in tqdm(range(6)):
  ts_entry=tss[i]
  forecast_entry=forecast[i]
  plot_prob_forecasts(ts_rntry,forecast_entry)

In [ ]:
from gluonts.evaluation import Evaluator
evaluator=Evaluator(quantiles=[0.1,0.5,0.9])
agg_metrics,item_metrics=evaluator(iter(tss),iter(forecasts), num_series=len(df_test[0:7]))

In [ ]:
item_metrics